In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [5]:
soup = BeautifulSoup(website_url, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":906439794,"wgRevisionId":906439794,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June",

In [6]:
neighborhood = soup.find('table', class_ = 'wikitable')
neighborhood_rows = neighborhood.find_all('tr')

In [7]:
information = []
for row in neighborhood_rows:
    info = row.text.split('\n')[1:-1] # remove empty str (the first and last items)
    information.append(info)
    
information[0:20] # Preview First 20 Rows

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District']]

In [8]:
neighbor_df = pd.DataFrame(information[1:], columns=information[0])
# where information[1:] contains each row of neighborhoods
# and columns = information[0] gives the column names


# Re-spell Neighbourhood as Neighborhood
neighbor_df = neighbor_df.rename(columns={neighbor_df.columns[2]: "Neighborhood" })


neighbor_df.head(20)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [9]:
neighbor_df = neighbor_df[neighbor_df.Borough != 'Not assigned']

neighbor_df.reset_index(drop=True, inplace=True)

neighbor_df.head(20)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [10]:
grouped = neighbor_df.groupby(['Postcode']) # group by Postcode


# combine the neighborhoods grouped by postcode and into a new df
neighborhood_grouped = grouped['Neighborhood'].apply(lambda x: x.sum()) 
# adds spaces and commas between neighborhoods
neighborhood_grouped = grouped['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
# matches a borough to each postcode
borough_grouped = grouped['Borough'].apply(lambda x: set(x).pop())
# turn borough_grouped and neighborhood_grouped into dataframes
borough = borough_grouped.to_frame()
neighborhood = neighborhood_grouped.to_frame()
#combine the dataframe borough and the dataframe neighborhood into one dataframe
grouped_final = borough.merge(neighborhood, on="Postcode")

grouped_final

,Borough,Neighborhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"


In [11]:
print('The number of rows and columns in this final grouped dataframe is',grouped_final.shape)


The number of rows and columns in this final grouped dataframe is (103, 2)


In [14]:
geospatial_data = pd.read_csv('http://cocl.us/Geospatial_data')

geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
geospatial_data = geospatial_data.rename(columns={geospatial_data.columns[0]: "Postcode" })

In [16]:
full_table = grouped_final.merge(geospatial_data, on = 'Postcode')

full_table.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [17]:
#Explore and cluster the neighborhoods of Toronto
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /Users/gonsolinx/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.7.10               |           py37_0         3.0 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0
  geopy              conda-forge/noarch::geopy-1.20.0-py_0

The following packages will be UPDATED:

  conda                                        4.7.8-py37_0 --> 4.7.10-py37_0



conda-4.7.10         | 3.0 MB    | ########################

In [18]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [19]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(full_table['Borough'].unique()),
        full_table['Neighborhood'].shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [20]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [21]:
# create a map of Toronto
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

#add neighborhood markers to the Toronto map
for lat, long, bor, neigh in zip(full_table['Latitude'], full_table['Longitude'], 
                                 full_table['Borough'], full_table['Neighborhood']):
    label = '{}, {}'.format(neigh, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 7, 
        popup = label,
        color = 'red',
        fill = True,
        fill_color = 'white',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)
        
map_toronto

In [22]:
list_boroughs = full_table['Borough'].unique()
list_boroughs

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [23]:
def borough_loc(list_of_places):
    for place in list_of_places:
        address = (place + ", Ontario, Canada")
        geolocator = Nominatim(user_agent="TO_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('{''}, {}, {},'.format(place,latitude,longitude))

borough_loc(list_boroughs)

Scarborough, 43.773077, -79.257774,
North York, 43.7708175, -79.4132998,


GeocoderTimedOut: Service timed out

In [24]:
import numpy as np

boroughs = ['Scarborough', 43.773077, -79.257774,
'North York', 43.7708175, -79.4132998,
'East York', 43.6913391, -79.3278212,
'East Toronto', 43.653963, -79.387207,
'Central Toronto', 43.653963, -79.387207,
'Downtown Toronto', 43.655115, -79.380219,
'York', 44.0007518, -79.4372217,
'West Toronto', 43.653963, -79.387207,
"Queen's Park", 43.6599803, -79.3903686,
'Mississauga', 43.590338, -79.645729,
'Etobicoke', 43.6435559, -79.5656326]

boroughs_df = pd.DataFrame(np.array(boroughs).reshape(11,3), columns = ["Borough","Latitude","Longitude"])

boroughs_df

,Borough,Latitude,Longitude
0,Scarborough,43.773077,-79.257774
1,North York,43.7708175,-79.4132998
2,East York,43.6913391,-79.3278212
3,East Toronto,43.653963,-79.387207
4,Central Toronto,43.653963,-79.387207
5,Downtown Toronto,43.655115,-79.380219
6,York,44.0007518,-79.4372217
7,West Toronto,43.653963,-79.387207
8,Queen's Park,43.6599803,-79.3903686
9,Mississauga,43.590338,-79.645729


In [25]:
boroughs_df.dtypes

boroughs_df["Latitude"] = pd.to_numeric(boroughs_df["Latitude"])
boroughs_df["Longitude"] = pd.to_numeric(boroughs_df["Longitude"])

In [26]:
# create a map of Toronto
map_toronto_boroughs = folium.Map(location = [43.653963, -79.387207], zoom_start = 10)

#add neighborhood markers to the Toronto map
for lat, long, bor in zip(boroughs_df['Latitude'], boroughs_df['Longitude'], 
                                 boroughs_df['Borough']):
    label = '{}'.format(bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 7, 
        popup = label,
        color = 'red',
        fill = True,
        fill_color = 'white',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto_boroughs)
        
map_toronto_boroughs

In [27]:
full_table['Borough'].value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

In [28]:
# The geographical coordinates for Downtown Toronto
DT_lat = boroughs_df.iloc[5,1]
DT_long = boroughs_df.iloc[5,2]

# The dataframe that contains all the Downtown Toronto neighborhoods
DT_df = full_table[full_table['Borough'] == 'Downtown Toronto'].reset_index(drop = True)

# create map of Downtown Toronto neighborhoods using latitude and longitude values
map_DT = folium.Map(location=[DT_lat, DT_long], zoom_start=12.4)

# add markers to map
for lat, lng, label in zip(DT_df['Latitude'], DT_df['Longitude'], DT_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_DT)  
    
map_DT